# 处理美图相关的原图与遮罩图

In [1]:
# 导入 PIL 库
from PIL import Image
import numpy as np
import os
from tqdm import tqdm


def transform_mask(img: Image.Image):
    # 获取图片的宽度和高度
    width, height = img.size
    # 创建一个新的图片对象，模式为 单色
    new_img = Image.new('L', (width, height))
    # 遍历图片的每个像素
    for x in range(width):
        for y in range(height):
            # 获取当前像素的 RGBA 值
            r, g, b, a = img.getpixel((x, y))
            if a > 128:
                new_img.putpixel((x, y), 255)
            # 如果像素透明，将新图片的对应像素设为黑色
            else:
                new_img.putpixel((x, y), 0)
    # 返回新图片
    return new_img

# 定义一个函数，接受两个关键词和一个文件夹路径作为参数
def process_images(keyword_a, keyword_b, folder_path):
 # 创建 orgin 和 gt 文件夹
    orgin_path = os.path.join(folder_path, "orgin")
    gt_path = os.path.join(folder_path, "gt")
    os.makedirs(orgin_path, exist_ok=True)
    os.makedirs(gt_path, exist_ok=True)
    ll = os.listdir(folder_path)

    # 遍历文件夹中的所有文件
    for file in tqdm(ll, total=len(ll), desc="process_images"):
    # 获取文件的完整路径
        file_path = os.path.join(folder_path, file)
        # 如果文件是图片文件
        if file.endswith(".png") or file.endswith(".jpg") or file.endswith(".jpeg"):
        # 如果文件名包含关键词 A，将其复制到 orgin 文件夹中
            if keyword_a in file:
                orgin_file = os.path.join(orgin_path, file)
                # 将原图转为RGB模式保存，为 jpg 格式
                Image.open(file_path).convert("RGB").save(orgin_file.replace("png", "jpg"))
        # 如果文件名包含关键词 B，将其处理后保存到 gt 文件夹中
            elif keyword_b in file:
                # gt 文件要和 orgin 文件保持一致的文件名
                gt_file = os.path.join(gt_path, file.replace(keyword_b, keyword_a))
                # 打开图片文件
                img = Image.open(file_path)
                new_img = transform_mask(img)
                new_img.save(gt_file)

# 将图片从遮罩中抠离出来
def get_crop_image_from_mask(image_name, mask_name, output_name):
    image = Image.open(image_name)
    mask = Image.open(mask_name)
    image_array = np.array(image)
    # 将遮罩中的白色部分转为黑白透明
    new_mask_array = transform_mask(mask)
    new_mask_array.show()
    new_mask_array = np.array(new_mask_array)
    # 遍历蒙版，如果是黑色，将其设为透明，如果是白色，则继承蒙版的透明度
    for x in range(new_mask_array.shape[0]):
        for y in range(new_mask_array.shape[1]):
            if new_mask_array[x, y, 0] == 0:
                image_array[x, y, 3] = 0
            else:
                image_array[x, y, 3] = new_mask_array[x, y, 3]
    # 将蒙版转为图片对象
    image = Image.fromarray(image_array)
    image.save(output_name, "png")


# get_crop_image_from_mask("/Users/hashiro/Downloads/data/treasures/meitu2/meitu88569535.png",
#                          "/Users/hashiro/Downloads/data/treasures/meitu2/orgin88569535.png",
#                         "/Users/hashiro/Downloads/data/treasures/meitu2/output33.png")

In [2]:
# 处理美图原图与遮罩图
process_images("meitu", "orgin", "/Users/hashiro/Downloads/data/treasures/meitu2/")
# process_images("meitu", "orgin", "/Users/hashiro/Downloads/data/treasures/test/")

process_images:   1%|          | 1/193 [00:01<05:33,  1.74s/it]


KeyboardInterrupt: 

# 处理深度图

In [2]:
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

from transformers import DPTFeatureExtractor, DPTForDepthEstimation
import torch
import numpy as np
from PIL import Image


print(torch.__version__)

device = "mps"

feature_extractor = DPTFeatureExtractor.from_pretrained("Intel/dpt-large",cache_dir='./model')
model = DPTForDepthEstimation.from_pretrained("Intel/dpt-large", cache_dir='./model')

def process_image_depth(image):
    image = image.convert('RGB')
    # prepare image for the model
    encoding = feature_extractor(image, return_tensors="pt").to(device)
    model.to(device)
    
    # forward pass
    with torch.no_grad():
       outputs = model(**encoding)
       predicted_depth = outputs.predicted_depth
    
    # interpolate to original size
    prediction = torch.nn.functional.interpolate(
                        predicted_depth.unsqueeze(1),
                        size=image.size[::-1],
                        mode="bicubic",
                 ).squeeze()
    output = prediction.cpu().numpy()
    formatted = (output * 255 / np.max(output)).astype('uint8')
    return formatted

# 将某个目录下的所有图片转换为深度图
def process_images_depth(input_dir, output_dir):
    # 如果文件夹不存在则创建
    os.makedirs(output_dir, exist_ok=True)
    for filename in tqdm.tqdm(os.listdir(input_dir)):
        image = Image.open(os.path.join(input_dir, filename))
        depth = process_image_depth(image)
        Image.fromarray(depth).save(os.path.join(output_dir, filename))
        


2.0.0


Some weights of DPTForDepthEstimation were not initialized from the model checkpoint at Intel/dpt-large and are newly initialized: ['neck.fusion_stage.layers.0.residual_layer1.convolution1.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.weight', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
import tqdm
# 处理图片深度图
process_images_depth('/Users/hashiro/Downloads/data/treasures/test/orgin', '/Users/hashiro/Downloads/data/treasures/test/depth')

100%|██████████| 15/15 [00:10<00:00,  1.48it/s]
